# **Ingestion of Data from API**
## This program collects appropriate data from an API and cleans it to suit and finally stores it into tables in the database.


In [145]:
%%capture
from dotenv import load_dotenv
import os

load_dotenv()

## Set up libraries and packages ##

In [164]:
#@title set up libraries
## set up libraries and packages needed

import pandas as pd
import yfinance as yf
import psycopg2


## Set up connection to the database ##

In [147]:
%%capture
# @title Set up connection to the database
def connect():
    connection = psycopg2.connect(
        host = os.environ.get('thost'),
        database = os.environ.get('tdatabase'),
        user = os.environ.get('tuser'),
        password = os.environ.get('tpassword'),
        port = '5432',
        options='-c search_path=student')
    return connection

connection = connect()


## Get the last date and time the table was updated ##

In [148]:
%%capture
# @title Get last date and time the table was updated
table = os.environ.get('tquery')
query = f"select max(period) as last_date from {table};"
get_last_date = pd.read_sql_query(query,connection)


In [149]:
print(get_last_date['last_date'])

0   2023-08-29 15:30:00
Name: last_date, dtype: datetime64[ns]


# Set timeframe in which to extract new data from API:
#### For first time - when the table is empty then max date is within last 60 days
#### Otherwise can use the last timestamp from the table

In [150]:
%%capture
# @title Set timeframe to extract data from API:

# Due to limitations of the API we can only download historical the last 60 days worth of data to use for populating the tables

# if table empty the set to max 60 days
current_date = pd.Timestamp.now().date()
# what is 60 days ago?
def_earliest_date = (current_date - pd.DateOffset(days=59)).date()
# set date if empty - only if table is empty
get_last_date.fillna(def_earliest_date, inplace = True)
# setup start date
earliest_date = get_last_date['last_date'][0]

## get the newest data from the api ##

In [ ]:
# @title Get Data for 3 chosen Stocks

data = yf.download("BARC.L MTRO.L LLOY.L", start=earliest_date, end=None, group_by='tickers', interval='30m')
# data


# **Clean the Data**

### format columns

In [152]:
stockdf = pd.DataFrame(data.stack(0).reset_index().rename(columns={'level_1': 'Ticker'}))

In [ ]:
# stockdf.head()

### remove unwanted columns ###

In [154]:
cols_to_remove = ['Adj Close', 'Volume']
stockdf2 = stockdf.drop(columns=cols_to_remove)
# stockdf2

### format data ###

In [155]:
stockdf2['Ticker'] = stockdf2['Ticker'].str.rstrip(".L")

In [ ]:
#stockdf2.head()

In [157]:
# stockdf2.tail()

In [158]:
## optionally for large data set - when running for first time save as .csv file
# not needed now
# stockdf2.to_csv('stocks.csv')

# Filter rows #
*Now* choose filter stock table where date > maxdatetime - So only keeping new records.





In [159]:
%%capture
filtered_view = stockdf2.loc[
    pd.to_datetime(stockdf2['Datetime']) > pd.to_datetime(get_last_date['last_date'][0])
]


# Check how many rows will be inserted into Database #

In [160]:
# print(stockdf2['Datetime'].count())

print(filtered_view['Datetime'].count(), 'rows will be inserted')

3 rows will be inserted


# Update Database with the filtered data

In [161]:
cursor = connection.cursor()


In [162]:
# insert into database
for index, row in filtered_view.iterrows():
    insert_query = f"INSERT INTO {table}(period, ticker, close, high, low, open) VALUES (%s, %s, %s, %s, %s, %s)"
    values = tuple(row)
    cursor.execute(insert_query,values)


# Commit and close connections

In [ ]:
# Commit the changes and close the cursor and connection
connection.commit()
cursor.close()
connection.close()

print('*** update complete ***')